# How to build Agents with Weaviate and Letta!

This repository will illustrate the core concepts behind Letta, a new startup built from the success of MemGPT from researchers at UC Berkeley.

Letta helps you build Agents with **meta memory**.

In addition to the prompt sent to an LLM, the LLM will also be provided with a short description of it's internal knowledge about the user and their interests derived from previous chats.

There are 3 parts to this notebook:

## 1. Getting Started with Letta Agents

An illustration of the basic setup requirements and a simple example of an Agent with internal memory.

## 2. Upload Weaviate Search Tool to Letta

Wrap Weaviate search in a Python function and upload it to the Letta Tool server.

## 3. Letta Agent with Weaviate

Construct a Letta Agent that chats with Weaviate's blog posts indexed in Weaviate. Observe how the Letta Agent develops an internal model of the chat user and the blog posts through these interactions.

<img src="architectural-visual.png" alt="visual" width="30%" height="auto"/>


# 1. Getting Started with Letta Agents

### Primer: Install Letta Server

`$pip install -U letta`

Start local server:

`$letta server`

### You can then create a Letta Agent in the GUI

We will also see how to do this with Python APIs in the notebook.

<img src="letta-create-agent.png" alt="Create Letta Agent GUI" width="60%" height="auto"/>



### Connect to Letta Client

In [25]:
from letta import create_client 

# connect to the letta server
letta_client = create_client(base_url="http://localhost:8283")

# get this from your local GUI
my_letta_agent_id = "agent-7abd9578-7e64-4416-ba71-5a3f908d0bea"

# send a message to the agent
response = letta_client.send_message(
    agent_id=my_letta_agent_id, 
    role="user", 
    message="How are you? Can you tell me about how HNSW works?"
)


In [3]:
# Print the internal monologue
print("Internal Monologue:")
print(response.messages[0].internal_monologue)
print()

# Print the message sent
print("Message Sent:")
print(response.messages[1].function_call.arguments)
print()

# Print the function return status
print("Function Return Status:")
print(f"Status: {response.messages[2].status}")
print(f"Time: {response.messages[2].function_return}")
print()

# Print usage statistics
print("Usage Statistics:")
print(f"Completion tokens: {response.usage.completion_tokens}")
print(f"Prompt tokens: {response.usage.prompt_tokens}") 
print(f"Total tokens: {response.usage.total_tokens}")
print(f"Step count: {response.usage.step_count}")

Internal Monologue:
Responding to the user's request for information on HNSW. Given his background, I'm providing a concise yet in-depth description. Now waiting for his follow-up.

Message Sent:
{
  "message": "I'm doing great, thank you! As for HNSW, Hierarchical Navigable Small World graphs, it's a state-of-the-art method for nearest neighbor search in high dimensional spaces. The main idea is that during the indexing stage, it builds a graph with a small-world property that allows for quick traversal. The graph's vertices are the data items, and each vertex has a set of connections on different layers - the lowest layer connects to the nearest items, while the higher layers have connections that span larger distances. This hierarchy significantly accelerates the search process because it can quickly navigate to the right area in earlier, higher layers, then fine-tune on lower layers. Would you like a more technical deep dive, perhaps focusing on the insertion algorithm or the searc

In [4]:
response = letta_client.send_message(
    agent_id=my_letta_agent_id, 
    role="user", 
    message="Please explain how Product Quantization saves the memory needed to store vector embeddings in a database management system."
)

# Print the internal monologue
print("Internal Monologue:")
print(response.messages[0].internal_monologue)
print()

# Print the message sent
print("Message Sent:")
print(response.messages[1].function_call.arguments)
print()

# Print the function return status
print("Function Return Status:")
print(f"Status: {response.messages[2].status}")
print(f"Time: {response.messages[2].function_return}")
print()

# Print usage statistics
print("Usage Statistics:")
print(f"Completion tokens: {response.usage.completion_tokens}")
print(f"Prompt tokens: {response.usage.prompt_tokens}") 
print(f"Total tokens: {response.usage.total_tokens}")
print(f"Step count: {response.usage.step_count}")

Internal Monologue:
Explaining Product Quantization. The user seems to be interested in database management systems, so focusing on memory efficiency.

Message Sent:
{
  "message": "Product Quantization (PQ) is a technique for high-dimensional vectors compression that notably boosts the efficiency of nearest neighbor searches. It works by dividing the high-dimensional vector space into smaller, manageable subspaces. Then, it independently quantizes each subspace into 'codewords' derived from a limited 'codebook'. When a vector is stored, it's mapped to a combination of these codewords from the different subspaces. The high-dimensional vector retrieval now becomes retrieving the local codewords and assembling them. Thus, instead of storing the high-dimensional vector in full, we store the compact indices referencing the codebook. This reduces the storage requisite dramatically without gravely compromising the search precision. Essentially, PQ trades a bit of accuracy for the benefit of 

In [12]:
response = letta_client.send_message(
    agent_id=my_letta_agent_id, 
    role="user", 
    message="Can you tell me what kind of database topics I am the most interested in?"
)

print(response)

{
    "messages": [
        {
            "id": "message-0b60401b-8b3d-4349-86c6-69c666b6b17f",
            "date": "2024-10-28T02:03:16+00:00",
            "message_type": "internal_monologue",
            "internal_monologue": "Again Chad is asking about his database interests. Let's review the conversation history once more to ensure we're not missing anything."
        },
        {
            "id": "message-0b60401b-8b3d-4349-86c6-69c666b6b17f",
            "date": "2024-10-28T02:03:16+00:00",
            "message_type": "function_call",
            "function_call": {
                "name": "conversation_search",
                "arguments": "{\n  \"query\": \"database\",\n  \"request_heartbeat\": true\n}",
                "function_call_id": "call_kuc4xYiVcX3NHPRwoLOF1qud"
            }
        },
        {
            "id": "message-6a86ac68-c571-4bf4-8b42-81a07bb05b96",
            "date": "2024-10-28T02:03:16+00:00",
            "message_type": "function_return",
            

# 2. Upload Weaviate Tool to Letta

In [13]:
import os

os.environ["OPENAI_API_KEY"] = "sk-foobar"
os.environ["WEAVIATE_URL"] = "https://weaviate-foobar.c0.us-east1.gcp.weaviate.cloud"
os.environ["WEAVIATE_API_KEY"] = "weaviate-foobar"

In [14]:

import weaviate
from weaviate.classes.init import Auth


# Connect to Weaviate Cloud
weaviate_client = weaviate.connect_to_weaviate_cloud(
    cluster_url=os.environ["WEAVIATE_URL"],
    auth_credentials=Auth.api_key(os.environ["WEAVIATE_API_KEY"]),
    headers={
        "X-OpenAI-Api-Key": os.environ["OPENAI_API_KEY"]
    }
)

print(weaviate_client.is_ready())

/var/folders/41/8dp_379x15d8zz4ppsjthdw40000gn/T/ipykernel_77675/2823592656.py:6: ResourceWarning: unclosed <ssl.SSLSocket fd=81, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('10.0.0.119', 50331), raddr=('35.201.124.182', 443)>
  weaviate_client = weaviate.connect_to_weaviate_cloud(


True


In [41]:
def search_weaviate_collection(
    self,
    search_query: str,
    ):
    """
    This tool queries an external database collection
    named by the parameter {collection_name} to find the most semantically similar items to the query.

    Args: 
        collection_name (str): The name of the database collection
        search_query (str): The search query

    Returns: 
        search_results (str): The results from the search engine.
    """
    import weaviate
    from weaviate.classes.init import Auth
    import os
    weaviate_client = weaviate.connect_to_weaviate_cloud(
        cluster_url=os.environ["WEAVIATE_URL"],
        auth_credentials=Auth.api_key(os.environ["WEAVIATE_API_KEY"]),
        headers={
            "X-OpenAI-Api-Key": os.environ["OPENAI_API_KEY"]
        }
    )
    # ToDo, need to figure out how to dynamically create these collections
    weaviate_collection = weaviate_client.collections.get("WeaviateBlogs")
    query_result = weaviate_collection.query.hybrid(
        query=search_query,
        alpha=0.5,
        limit=5
    )
    weaviate_client.close()
    results = query_result.objects
    formatted_results = "\n".join(
        [f"[Search Result {i+1}] {str(result.properties)}" for i, result in enumerate(results)]
    )
    return formatted_results

In [42]:
results_demo = search_weaviate_collection(
    None,  # Remove self since this is just a function, not a method
    "What is HNSW?"
);

print("\033[92mResults demo:\033[0m");
print(results_demo);

Results demo:
[Search Result 1] {'content': 'Due to its relatively high memory footprint, HNSW is only cost-efficient in high-throughput scenarios. However, HNSW is inherently optimized for in-memory access. Simply storing the index or vectors on disk or memory-mapping the index kills performance. This is why we will offer you not just one but two memory-saving options to index your vectors without sacrificing latency and throughput. In early 2023, you will be able to use Product Quantization, a vector compression algorithm, in Weaviate for the first time.'}
[Search Result 2] {'content': '---\ntitle: HNSW+PQ - Exploring ANN algorithms Part 2.1\nslug: ann-algorithms-hnsw-pq\nauthors: [abdel]\ndate: 2023-03-14\ntags: [\'research\']\nimage: ./img/hero.png\ndescription: "Implementing HNSW + Product Quantization (PQ) vector compression in Weaviate."\n---\n![HNSW+PQ - Exploring ANN algorithms Part 2.1](./img/hero.png)\n\n<!-- truncate -->\n\nWeaviate is already a very performant and robust [

In [45]:
weaviate_search_tool = letta_client.create_tool(
    search_weaviate_collection,
    tags=["weaviate", "search"]
)

In [46]:
letta_client.list_tools()

[Tool(description=None, source_type='python', module=None, user_id='user-00000000-0000-4000-8000-000000000000', id='tool-0571e78e-c70a-4874-82d1-bdc77f3f0745', name='search_weaviate_collection', tags=['weaviate', 'search'], source_code='def search_weaviate_collection(\n    self,\n    search_query: str,\n):\n    """\n    This tool queries an external database collection\n    named by the parameter `collection_name` to find the most semantically similar items to the query.\n\n    Args: \n        collection_name (str): The name of the database collection\n        search_query (str): The search query\n\n    Returns: \n        search_results (str): The results from the search engine.\n    """\n    import weaviate\n    from weaviate.classes.init import Auth\n    import os\n    weaviate_client = weaviate.connect_to_weaviate_cloud(\n        cluster_url=os.environ["WEAVIATE_URL"],\n        auth_credentials=Auth.api_key(os.environ["WEAVIATE_API_KEY"]),\n        headers={\n            "X-OpenAI-A

You can also see the available tools on the Letta Server by going to "Tool Builder"

<img src="letta-tools.png" alt="Letta Tool Viewer" width="60%" height="auto"/>


You can even update the functions in your tools on the GUI!

<img src="letta-update-tools.png" alt="Edit Letta Tools" width="40%" height="auto"/>


# 3. Letta Agent with Weaviate

In [47]:
from letta import LLMConfig, EmbeddingConfig
# set default llm config for agents
letta_client.set_default_llm_config(
    LLMConfig.default_config(model_name="letta")
)

# set default embedding config for agents
letta_client.set_default_embedding_config(
    EmbeddingConfig.default_config(model_name="letta")
)

agent_state = letta_client.create_agent(
    name="newest-weaviate-blogs-agent", 
    tools=[weaviate_search_tool.name], 
)

new_agent_id = agent_state.id

In [48]:
response = letta_client.send_message(
    agent_id=new_agent_id,
    role="user",
    message="Can you please tell me about Weaviate's Vector Indexing features?"
)

print(response)

{
    "messages": [
        {
            "id": "message-88154604-59aa-4fd0-9d1e-807f8f47cbd6",
            "date": "2024-10-28T02:13:33+00:00",
            "message_type": "internal_monologue",
            "internal_monologue": "The user is looking for information about Weaviate's Vector Indexing features. Let's perform a search in the external database to fetch the most accurate response."
        },
        {
            "id": "message-88154604-59aa-4fd0-9d1e-807f8f47cbd6",
            "date": "2024-10-28T02:13:33+00:00",
            "message_type": "function_call",
            "function_call": {
                "name": "search_weaviate_collection",
                "arguments": "{\n  \"search_query\": \"Weaviate Vector Indexing features\",\n  \"request_heartbeat\": true\n}",
                "function_call_id": "call_d7Tu2SMdFWwZ0XzefTAsr6hd"
            }
        },
        {
            "id": "message-0f6ba847-8183-4640-95ab-8a238ccbc32e",
            "date": "2024-10-28T02:13:35+0

In [49]:
response = letta_client.send_message(
    agent_id=new_agent_id,
    role="user",
    message="Can you please tell me about how Product Quantization works?"
)

print(response)

{
    "messages": [
        {
            "id": "message-ba06f6f6-a885-4cb0-b51c-a48acc041f57",
            "date": "2024-10-28T02:13:50+00:00",
            "message_type": "internal_monologue",
            "internal_monologue": "The user is curious about how Product Quantization works. Searching for a condensed response in our external database."
        },
        {
            "id": "message-ba06f6f6-a885-4cb0-b51c-a48acc041f57",
            "date": "2024-10-28T02:13:50+00:00",
            "message_type": "function_call",
            "function_call": {
                "name": "search_weaviate_collection",
                "arguments": "{\n  \"search_query\": \"Product Quantization working\",\n  \"request_heartbeat\": true\n}",
                "function_call_id": "call_1VvD849HDYYYKC9TOE9iiYzU"
            }
        },
        {
            "id": "message-412f7c1a-38af-4135-8343-629970a639f3",
            "date": "2024-10-28T02:13:51+00:00",
            "message_type": "function_retur

In [50]:
response = letta_client.send_message(
    agent_id=new_agent_id,
    role="user",
    message="Can you please tell me about how Product Quantization works?"
)

print(response)

{
    "messages": [
        {
            "id": "message-1cde7e9f-e635-4ffd-bc52-6668b73f8445",
            "date": "2024-10-28T02:14:07+00:00",
            "message_type": "internal_monologue",
            "internal_monologue": "The user is curious about how Product Quantization works. I will search the external Weaviate collection for a detailed yet concise explanation."
        },
        {
            "id": "message-1cde7e9f-e635-4ffd-bc52-6668b73f8445",
            "date": "2024-10-28T02:14:07+00:00",
            "message_type": "function_call",
            "function_call": {
                "name": "search_weaviate_collection",
                "arguments": "{\n  \"search_query\": \"Product Quantization working\",\n  \"request_heartbeat\": true\n}",
                "function_call_id": "call_lwbmxGjbkWdVPNWXjPVhtCb4"
            }
        },
        {
            "id": "message-3472d4c3-0455-4ca4-a862-2b5e0328e013",
            "date": "2024-10-28T02:14:08+00:00",
            "mess

In [51]:
response = letta_client.send_message(
    agent_id=new_agent_id,
    role="user",
    message="Can you please tell me about what I've been learning about Weaviate?"
)

print(response)

{
    "messages": [
        {
            "id": "message-621443d9-55d4-4e84-9818-e88987240191",
            "date": "2024-10-28T02:14:35+00:00",
            "message_type": "internal_monologue",
            "internal_monologue": "The user wants to recall what he's learned about Weaviate. I'll use the conversation_search function to find relevant parts of our past conversations."
        },
        {
            "id": "message-621443d9-55d4-4e84-9818-e88987240191",
            "date": "2024-10-28T02:14:35+00:00",
            "message_type": "function_call",
            "function_call": {
                "name": "conversation_search",
                "arguments": "{\n  \"query\": \"Weaviate\",\n  \"request_heartbeat\": true\n}",
                "function_call_id": "call_nwSAud5X36ty7pCF01Zzo6nV"
            }
        },
        {
            "id": "message-9daaf309-a1a6-4fa0-833c-d5ddd18bf53a",
            "date": "2024-10-28T02:14:35+00:00",
            "message_type": "function_return"

# Thanks for checking out our first Weaviate and Letta Notebook!

### Keep up with us on X to be notified of future updates!

- [Charles Packer](https://x.com/charlespacker) 
- [Sarah Wooders](https://x.com/sarahwooders)
- [Erika Cardenas](https://x.com/ecardenas300)
- [Connor Shorten](https://x.com/CShorten30)